This script tests the ENG111-specific model which only includes the course-specific & instructor-related admin & subject-specific predictors.		

In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt

results_dir = "~\\Box Sync\\Clickstream\\evaluation_results\\full\\"

In [2]:
sn_dict = {"Blue Ridge": "BRCC",
           "Central Virginia": "CVCC",
           "Dabney S. Lancaster": "DSLCC",
           "Danville": "DCC",
           "Eastern Shore": "ESCC",
           "Germanna": "GCC",
           'J. Sargeant Reynolds': "JSRCC",
           'John Tyler': "JTCC",
           "Lord Fairfax": "LFCC",
           "Mountain Empire": "MECC",
           "New River": "NRCC",
           "Northern Virginia": "NVCC",
           "Patrick Henry": "PHCC",
           "Paul D. Camp": "PDCCC",
           "Piedmont Virginia": "PVCC",
           "Rappahannock": "RCC",
           "Southside Virginia": "SSVCC",
           "Southwest Virginia": "SWVCC",
           "Thomas Nelson": "TNCC",
           "Tidewater": "TCC",
           "Virginia Highlands": "VHCC",
           "Virginia Western": "VWCC",
           "Wytheville": "WCC"}

In [3]:
df0 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\LMS_data_final_full_new.csv")
df1 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\course_specific_predictors_new.csv")
df2 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\term_specific_predictors_new.csv")
df3 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\full\\cluster_specific_predictors.csv")
df3 = df3.loc[:,['vccsid','strm','college','course','section'] + [e for e in df3.columns.values if e.endswith("HUM") or e.endswith("HUM_grade")]]
df4 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\full\\instructor_related_predictors.dta")
df = df0.merge(df1, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df2, how='inner', on=['vccsid','strm'])\
.merge(df3, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df4, how='inner', on=['vccsid','strm','college','course','section'])
df = df[df.course == "ENG_111"]
df.loc[:,'first_ind'] = 0

In [4]:
df0 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\LMS_data_final.dta")
df1 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\course_specific_predictors_new.csv")
df2 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\term_specific_predictors_new.csv")
df4 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\instructor_related_predictors.dta")
df5 = df0.loc[:,['vccsid','strm','college','course','section']].copy()
df_first = df0.merge(df1, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df2, how='inner', on=['vccsid','strm'])\
.merge(df4, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df5, how='inner', on=['vccsid','strm','college','course','section'])
df_first = df_first[df_first.course == "ENG_111"]
df_first.loc[:,'first_ind'] = 1

In [5]:
df = pd.concat([df, df_first], axis=0, join='outer').fillna(0)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [6]:
Counter(df.first_ind)

Counter({0: 25967, 1: 28265})

In [7]:
for v in [int(e) for e in np.unique(df.cip) if e != 0]:
    df.loc[:,'cip_'+str(v)] = (df.cip == v).astype(int)
for v in [int(e) for e in np.unique(df.degree_level) if e != 4]:
    df.loc[:,'degree_level_'+str(v)] = (df.degree_level == v).astype(int)
df = df.drop(['cip', 'degree_level'], axis=1)
df.loc[:,'college_new'] = df.college.apply(lambda x: sn_dict[x])
for sn in [e for e in sn_dict.values() if e != "BRCC"]:
    df.loc[:,'college_'+sn] = (df.college_new == sn).astype(int)
df = df.drop(['college_new'], axis=1)

In [8]:
test = df.describe().T
test[test['mean'] == 0]

,count,mean,std,min,25%,50%,75%,max
has_prereq_grade,54232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
lvl2_ind,54232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
prereq_grade,54232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
df = df.drop(['lvl2_ind', 'has_prereq_grade', 'prereq_grade'], axis=1)
predictors = [e for e in list(df.columns) if e not in {"grade",'vccsid','strm','college','course','section'}]
len(predictors)

148

In [18]:
assert pd.isnull(df).any().any() == False

In [19]:
df.shape

(54232, 154)

In [20]:
early_lms = [e for e in predictors if (e.endswith("_qtr1") or e.endswith("_qrt1")) and (e != "has_concurrent_qtr1") and (e.startswith("prior_") == False)]
concurrent_lms = [e for e in predictors if e.endswith("_qtr1c") or e.endswith("_qrt1c") or e == 'has_concurrent_qtr1']
historical_early_lms = [e for e in predictors if e.startswith("prior") and (e.endswith("_qrt1") or e.endswith("_qtr1"))]
historical_full_lms = [e for e in predictors if e.startswith("prior") and e.endswith("_qrt1") == False and e.endswith("_qtr1") == False]
all_lms = early_lms + concurrent_lms + historical_early_lms + historical_full_lms

In [21]:
predictors = [e for e in predictors if e not in set(all_lms)]
len(predictors)

98

In [22]:
predictor_category = pd.read_csv("~\\Box Sync\\Clickstream\\evaluation_results\\first\\predictor_category_table.csv")

In [28]:
predictor_df = pd.DataFrame({'predictor': predictors}).merge(predictor_category, how='left', on=['predictor'])
predictor_df = predictor_df[predictor_df.predictor_subcategory.apply(lambda x: x in {'Course-specific', 'Course-subject-specific', 'Instructor-related'})]
predictors = list(predictor_df.predictor) + ['first_ind']
len(predictors)-1

55

In [29]:
train_df = df[df.strm != 2212]
test_df = df[df.strm == 2212]
original_test_grade = np.array(test_df.grade)
train_df.loc[:,'grade'] = train_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
test_df.loc[:,'grade'] = test_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
print(train_df.shape,test_df.shape)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(45253, 154) (8979, 154)


In [30]:
def create_cv_folds(train, n_fold = 5):
    folds = []
    k_fold = StratifiedKFold(n_splits = n_fold, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.grade):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        X_1 = train_part.loc[:,predictors]
        y_1 = train_part.grade
        X_2 = test_part.loc[:,predictors]
        y_2 = test_part.grade
        folds.append([(X_1.copy(),y_1.copy()),(X_2.copy(),y_2.copy())])
    return folds

In [31]:
five_folds = create_cv_folds(train_df)

In [32]:
def cross_validation_RF(rf_model, folds):
    auc_by_fold = []
    for f in folds:
        X_1 = f[0][0]
        y_1 = f[0][1]
        X_2 = f[1][0]
        y_2 = f[1][1]
        rf_model.fit(X_1,y_1)
        y_2_pred = rf_model.predict_proba(X_2)[:,1]
        auc_by_fold.append(roc_auc_score(y_2,y_2_pred))
    return round(np.mean(auc_by_fold),4)  

In [33]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [34]:
### Using grid search to find the optimal maximum tree depth
auc_by_d=[]
for d in range(2,26):
    rf = RandomForestClassifier(n_estimators=200, criterion="entropy", 
                                max_depth=d,
                                random_state=0, n_jobs=20, max_features="auto",
                                class_weight = calc_cw(train_df.grade))
    auc = cross_validation_RF(rf, five_folds)
    auc_by_d.append(auc)
    print("Max_depth =", d)
    print("Mean CV AUC:", auc)
    print("")
plt.plot(range(2,26),auc_by_d)
plt.xlabel("Maximum Depth")
plt.ylabel("AUC")
plt.show()

Max_depth = 2
Mean CV AUC: 0.6096

Max_depth = 3
Mean CV AUC: 0.6188

Max_depth = 4
Mean CV AUC: 0.6244

Max_depth = 5
Mean CV AUC: 0.6278

Max_depth = 6
Mean CV AUC: 0.6302

Max_depth = 7
Mean CV AUC: 0.6339

Max_depth = 8
Mean CV AUC: 0.6367

Max_depth = 9
Mean CV AUC: 0.6398

Max_depth = 10
Mean CV AUC: 0.6415

Max_depth = 11
Mean CV AUC: 0.6424

Max_depth = 12
Mean CV AUC: 0.6443

Max_depth = 13
Mean CV AUC: 0.6448

Max_depth = 14
Mean CV AUC: 0.6451

Max_depth = 15
Mean CV AUC: 0.6452

Max_depth = 16
Mean CV AUC: 0.6449

Max_depth = 17
Mean CV AUC: 0.6452

Max_depth = 18
Mean CV AUC: 0.6448

Max_depth = 19
Mean CV AUC: 0.6437

Max_depth = 20
Mean CV AUC: 0.6422

Max_depth = 21
Mean CV AUC: 0.6422

Max_depth = 22
Mean CV AUC: 0.6408

Max_depth = 23
Mean CV AUC: 0.6391



KeyboardInterrupt: 

In [35]:
### Using grid search to find the optimal number of estimators (trees)
auc_by_n = []
for n in range(100,320,20):
    rf = RandomForestClassifier(n_estimators=n, criterion="entropy", 
                                max_depth=10,
                                random_state=0, n_jobs=-1, max_features="auto",
                                class_weight = calc_cw(train_df.grade))
    auc = cross_validation_RF(rf, five_folds)
    auc_by_n.append(auc)
    print("Number of Trees =", n)
    print("Mean CV AUC:", auc)
    print("")
plt.plot(range(100,320,20), auc_by_n)
plt.xlabel("Number of Trees")
plt.ylabel("AUC")
plt.show()

Number of Trees = 100
Mean CV AUC: 0.6409

Number of Trees = 120
Mean CV AUC: 0.6412

Number of Trees = 140
Mean CV AUC: 0.6412

Number of Trees = 160
Mean CV AUC: 0.6415

Number of Trees = 180
Mean CV AUC: 0.6415



KeyboardInterrupt: 

In [36]:
### Using grid search to find the optimal maximum number of features (trees)
auc_by_nf = []
max_nf = int(np.floor(2*np.sqrt(len(predictors))))
for nf in range(2,max_nf+1):
    rf = RandomForestClassifier(n_estimators=120, criterion="entropy", 
                                max_depth=10,
                                random_state=0, n_jobs=-1, max_features=nf,
                                class_weight = calc_cw(train_df.grade))
    auc = cross_validation_RF(rf, five_folds)
    auc_by_nf.append(auc)
    print("Max_features =", nf)
    print("Mean CV AUC:", auc)
    print("")
plt.plot(range(2,max_nf+1), auc_by_nf)
plt.xlabel("Maximum Number of Features")
plt.ylabel("AUC")
plt.show()

Max_features = 2
Mean CV AUC: 0.6385

Max_features = 3
Mean CV AUC: 0.64

Max_features = 4
Mean CV AUC: 0.6408

Max_features = 5
Mean CV AUC: 0.6402

Max_features = 6
Mean CV AUC: 0.6397

Max_features = 7
Mean CV AUC: 0.6412

Max_features = 8
Mean CV AUC: 0.6414



KeyboardInterrupt: 

In [37]:
rf = RandomForestClassifier(n_estimators=120, criterion="entropy",
                            max_depth=10,
                            random_state=0, n_jobs=-1, max_features=4,
                            class_weight = calc_cw(train_df.grade))
rf.fit(train_df.loc[:,predictors], train_df.grade)

RandomForestClassifier(bootstrap=True, class_weight={0: 1.4809211, 1: 1.0},
            criterion='entropy', max_depth=10, max_features=4,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=120, n_jobs=-1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [38]:
print("Random Forest:")
print("Overall AUC = {}".format(round(roc_auc_score(test_df.grade, rf.predict_proba(test_df.loc[:,predictors])[:,1]),4)))
y_test_pred_rf = rf.predict_proba(test_df.loc[:,predictors])[:,1]

Random Forest:
Overall AUC = 0.698


In [39]:
print("Random Forest:")
print("Non-first AUC = {}".format(round(roc_auc_score(test_df[test_df.first_ind == 0].grade, rf.predict_proba(test_df[test_df.first_ind == 0].loc[:,predictors])[:,1]),4)))
y_test_pred_rf_nonfirst = rf.predict_proba(test_df[test_df.first_ind == 0].loc[:,predictors])[:,1]

Random Forest:
Non-first AUC = 0.733


In [40]:
print("Random Forest:")
print("First AUC = {}".format(round(roc_auc_score(test_df[test_df.first_ind == 1].grade, rf.predict_proba(test_df[test_df.first_ind == 1].loc[:,predictors])[:,1]),4)))
y_test_pred_rf_first = rf.predict_proba(test_df[test_df.first_ind == 1].loc[:,predictors])[:,1]

Random Forest:
First AUC = 0.5182


In [41]:
# print("ABC vs. DF")
# print("AUC = {}".format(round(roc_auc_score(np.array(test_df.grade)[np.where(np.array(original_test_grade) != "W")[0]], 
#                                             rf.predict_proba(test_df.loc[:,predictors])[np.where(np.array(original_test_grade) != "W")[0],1]),4)))

In [42]:
# def find_optimal_threshold(p,r,t):
#     to_drop = np.union1d(np.where(pd.isnull(p[:-1]) == True)[0], np.where(pd.isnull(r[:-1]) == True)[0])
#     to_drop = np.union1d(to_drop, np.where(pd.isnull(t) == True)[0])
#     to_keep = np.setdiff1d(np.array(list(range(len(p)-1))), to_drop)
#     p,r,t = p[to_keep],r[to_keep],t[to_keep]
#     to_keep_2 = np.where(t < 0.8)[0]
#     p,r,t = p[to_keep_2],r[to_keep_2],t[to_keep_2]
#     f1 = 2*p*r/(p+r)
#     best_t = t[np.argmax(f1)]
#     best_t
#     return best_t

# def cross_validation(train, model):
#     threshold_list = []
#     auc_list = []
#     k_fold =  StratifiedKFold(n_splits = 10, random_state = 54321, shuffle=True)
#     for train_indices, test_indices in k_fold.split(train, train.grade):
#         train_part = train.iloc[train_indices,:]
#         test_part = train.iloc[test_indices,:]
#         X_1 = train_part.loc[:,predictors]
#         y_1 = train_part.grade
#         X_2 = test_part.loc[:,predictors]
#         y_2 = test_part.grade
#         model.fit(X_1,y_1)
#         p,r,t = precision_recall_curve(1-np.array(y_2), model.predict_proba(X_2)[:,0])
#         threshold_list.append(1-find_optimal_threshold(p,r,t))
#         auc = roc_auc_score(y_2, model.predict_proba(X_2)[:,1])
#         auc_list.append(auc)
#     print(threshold_list)
#     print(np.mean(auc_list), np.std(auc_list, ddof=1))
#     return gmean(threshold_list)

In [43]:
# best_threshold = cross_validation(train_df,rf)

In [44]:
# best_threshold

In [45]:
best_threshold = np.sort(y_test_pred_rf)[int(len(y_test_pred_rf) * (1-np.mean(train_df.grade)))-1]

In [46]:
def create_confusion_matrix_old(y_test_pred, threshold, fname):
    cm_arr = confusion_matrix(y_test, np.where(y_test_pred > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_DFW','Pred_ABC'], index=['Actual_DFW', 'Actual_ABC'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score for A/B/C = {}".format(round(2*p1*r1/(p1+r1),4)))
    print("F1 score for D/F/W = {}".format(round(2*p0*r0/(p0+r0),4))) 
    cm_df.to_csv(results_dir + fname + ".csv")
    y_test_pred_bin = np.where(y_test_pred > best_threshold, 1, 0)
    cm_dict = {}
    cm_dict['Pred_DFW'] = Counter(original_test_grade[np.where(y_test_pred_bin==0)[0]])
    cm_dict['Pred_ABC'] = Counter(original_test_grade[np.where(y_test_pred_bin==1)[0]])
    new_cm = pd.DataFrame.from_dict(cm_dict, orient='index').T.loc[['W','F','D','C','B','A'],['Pred_DFW','Pred_ABC']]
    new_cm.index = ["Actual_"+e for e in new_cm.index]
    new_cm.loc[:,''] = new_cm.sum(axis=1)
    new_cm.loc['',:] = new_cm.sum(axis=0)
    new_cm.to_csv(results_dir + fname + "_6x2.csv")
    return round(p1,4),round(r1,4),round(p0,4),round(r0,4),round(2*p1*r1/(p1+r1),4),round(2*p0*r0/(p0+r0),4)

In [47]:
y_test = np.array(test_df.grade)
print("F1 threshold = {}:\n".format(str(round(best_threshold,4))))
pr_rf = create_confusion_matrix_old(y_test_pred_rf, best_threshold, "RF_ENG111_all_cm_admin_subcategory1")
print(pr_rf)

F1 threshold = 0.5478:

            Pred_DFW  Pred_ABC        
Actual_DFW    1730.0    1924.0  3654.0
Actual_ABC    1081.0    4244.0  5325.0
              2811.0    6168.0  8979.0

F1 score for A/B/C = 0.7385
F1 score for D/F/W = 0.5352
(0.6881, 0.797, 0.6154, 0.4735, 0.7385, 0.5352)


In [48]:
def create_confusion_matrix(y_test_pred, threshold, fname, ind = 0):
    cm_arr = confusion_matrix(y_test[np.array(test_df.first_ind == ind)], np.where(y_test_pred > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_DFW','Pred_ABC'], index=['Actual_DFW', 'Actual_ABC'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score for A/B/C = {}".format(round(2*p1*r1/(p1+r1),4)))
    print("F1 score for D/F/W = {}".format(round(2*p0*r0/(p0+r0),4))) 
    cm_df.to_csv(results_dir + fname + ".csv")
    y_test_pred_bin = np.where(y_test_pred > best_threshold, 1, 0)
    cm_dict = {}
    cm_dict['Pred_DFW'] = Counter(original_test_grade[np.array(test_df.first_ind == ind)][np.where(y_test_pred_bin==0)[0]])
    cm_dict['Pred_ABC'] = Counter(original_test_grade[np.array(test_df.first_ind == ind)][np.where(y_test_pred_bin==1)[0]])
    new_cm = pd.DataFrame.from_dict(cm_dict, orient='index').T.loc[['W','F','D','C','B','A'],['Pred_DFW','Pred_ABC']]
    new_cm.index = ["Actual_"+e for e in new_cm.index]
    new_cm.loc[:,''] = new_cm.sum(axis=1)
    new_cm.loc['',:] = new_cm.sum(axis=0)
    new_cm.to_csv(results_dir + fname + "_6x2.csv")
    return round(p1,4),round(r1,4),round(p0,4),round(r0,4),round(2*p1*r1/(p1+r1),4),round(2*p0*r0/(p0+r0),4)

In [49]:
y_test = np.array(test_df.grade)
print("F1 threshold = {}:\n".format(str(round(best_threshold,4))))
pr_rf = create_confusion_matrix(y_test_pred_rf_nonfirst, best_threshold, "RF_ENG111_full_cm_admin_subcategory1")

F1 threshold = 0.5478:

            Pred_DFW  Pred_ABC        
Actual_DFW    1704.0     963.0  2667.0
Actual_ABC    1045.0    2456.0  3501.0
              2749.0    3419.0  6168.0

F1 score for A/B/C = 0.7098
F1 score for D/F/W = 0.6292


In [50]:
pr_rf

(0.7183, 0.7015, 0.6199, 0.6389, 0.7098, 0.6292)

In [51]:
print("F1 threshold = {}:\n".format(str(round(best_threshold,4))))
pr_rf = create_confusion_matrix(y_test_pred_rf_first, best_threshold, "RF_ENG111_first_cm_admin_subcategory1", 1)

F1 threshold = 0.5478:

            Pred_DFW  Pred_ABC        
Actual_DFW      25.0     962.0   987.0
Actual_ABC      36.0    1788.0  1824.0
                61.0    2750.0  2811.0

F1 score for A/B/C = 0.7818
F1 score for D/F/W = 0.0477


In [52]:
pr_rf

(0.6502, 0.9803, 0.4098, 0.0253, 0.7818, 0.0477)